<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

В данной работе необходимо было сделать разработку такого метода преобразования данных, чтобы по ним было сложно восстановить персональную информацию и качество моделей машинного обучения не ухудшилось.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('***')

In [3]:
df.sample(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
3937,0,29.0,42400.0,0,0
189,0,39.0,41200.0,0,0
2595,0,31.0,38800.0,1,0
3795,0,36.0,21700.0,2,0
4408,1,24.0,48000.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df.duplicated().sum()

153

In [6]:
df.drop_duplicates(inplace=True)

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** При умножении на обратимую матрицу, качество линейной регресии не изменится.

**Обоснование:** 
Формула расчета весов для линейной регрессии:

$$
w = (X^T X)^{-1} X^T y
$$
Представим новую матрицу признаков $X1$ как произведение старой $X$ на матрицу $P$ (обратимую):

$$
X1 = X * P
$$
Подставим новое значение X1 в формулу $w1$:

$$
w1 = ((X P)^T X P)^{-1} (X P)^T y
$$
Раскроем первое произведение $ (XP)^T $:

$$
w1 = (P^T X^T X P)^{-1} P^T X^T y
$$
Перегруппируем множители в скобках, чтобы это выглядело как произведение трех множителей:

$$
w1 = (P^T (X^T X) P)^{-1} P^T X^T y
$$
Раскроем скобки $ (P^T (X^T X) P)^{-1} $:

$$
w1 = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
Так как $P$ по условию обратимая, то произведение $(P^T)^{-1} P^T$ равно $E$ (единичная матрица):

$$
w1 = P^{-1} (X^T X)^{-1} E X^T y = P^{-1} (X^T X)^{-1} X^T y
$$
Можно заметить, что справа получилась формула для $w$:

$$
w1 = P^{-1} (X^T X)^{-1} X^T y = P^{-1} w
$$
Подставим новое значение весов 𝑤1 в формулу для предсказаний линейной регрессии. Предсказания модели расчитываются по формуле:

$$
a = Xw
$$
Подставим в эту формулу значения $X1$ и $w1$ для того, чтобы на их основе вычислить предсказания модели $a1$:

$$
a1 = X1w1 = XPP^{-1} w
$$
Так как $P$ по условию обратимая, то произведение $PP^{-1}$ снова равно $E$:

$$
a1 = XPP^{-1} w = X E w = X w = a
$$
Мы доказали, что предсказания $a1$ для матрицы признаков, умноженных на обратимую матрицу $P$ равны предсказаниям $a$. Следовательно, при умножении признаков на обратимую матрицу - качество линейной регрессии не изменится. В данном случае, параметры линейной регрессии в исходной задаче $w$ и в преобразованной $w1$ связаны следующим образом:

$$
w1 = P^{-1} w
$$

## Алгоритм преобразования

**Алгоритм**

    1.Создать случайную матрицу.
    2.Проверить ее на обратимость.
    3.Умножить на признаки.
    4.Найдем метрики обученной модели на исходных признаках и на признаках умноженных на нашу матрицу.
    5.Сравнить метрики между собой. Если они равны, то задача выполнена


## Проверка алгоритма

In [7]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

In [8]:
matrix = np.random.normal(size = (4,4))

In [9]:
matrix = np.linalg.inv(matrix)

In [10]:
model = LinearRegression()
model.fit(features, target)
normal_predictions = model.predict(features)
normal_r2_score = r2_score(target, normal_predictions)
print('R2 на исходных признаках:', normal_r2_score)

R2 на исходных признаках: 0.4302010044852067


In [11]:
transform_features = features @ matrix

In [12]:
model.fit(transform_features, target)
transform_predictions = model.predict(transform_features)
transform_r2_score = r2_score(target, transform_predictions)
print('R2 на преобразованных признаках:', transform_r2_score)

R2 на преобразованных признаках: 0.43020100448522136


In [13]:
normal_r2_score - transform_r2_score

-1.4654943925052066e-14

Качество линейной регрессии не изменилось, можно сделать вывод, что алгоритм преобразования данных работает правильно.